In [5]:
import os
import vedo
import numpy as np
import trimesh
from copy import copy
from scipy.special import sph_harm
import ipywidgets as widgets
from ipywidgets import interact

# Spherical harmonics

$$\textbf{s}_0=S^2$$

The spherical harmonics are defined by the following formula

$$Y_{lm}(\theta, \phi)=\sqrt{\frac{2l+1}{4\pi}}P_l^m(\cos\theta)e^{im\phi},$$

where the $P_l^m$ are the associated Legendre polynomials. They satisfy the following equation:

$$r^2\nabla^2Y_{lm}(\theta, \phi)=l(l+1)Y_{lm}(\theta, \phi)$$

In other words, the Laplacian operator $\nabla^2$ can be diagonalized by using this basis. The associated matrix for the first 7 spherical harmonics is:

$$\begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 2 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 2 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 2 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 6 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 6 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 6\\
\end{pmatrix}$$

The deformation is performed along the radial direction at each point, and with an amplitude given by the following expansion:

$$ \textbf{d}(\theta,\phi) =\bigg( \sum_{l=0}^{\infty}\sum_{m=-l}^{l}a_{lm}Y_{lm}(\theta, \phi)\bigg) \hat{\textbf{r}}$$

### A local deformation

The expansion of the Dirac delta is given by:

$$\delta(\theta,\phi)=\sum_{l=0}^{\infty}\sum_{m=-l}^{l}Y_{lm}(0, 0)Y_{lm}(\theta, \phi)$$

Would this be useful to create a local deformation (by truncating this expansion)?

### How to perform content and style deformations

One possibility is to generate a basis of spherical harmonics that spans the subspace of possible content deformations; and another that spans the subspace of style deformations.

$$ \textbf{d}_c(\theta,\phi)\in span(\mathcal{B}_c)$$
$$ \textbf{d}_s(\theta,\phi)\in span(\mathcal{B}_s)\subseteq span(\mathcal{B}_c)^{\bot}$$

The style deformation is such that it varies with time in a periodical manner. Therefore, the overall deformation will be of the form

$$ \textbf{d}(\theta,\phi) = \textbf{d}_c(\theta,\phi) + A(t)\textbf{d}_s(\theta,\phi) $$ with $$A(t)=A(t+2\pi)$$


For example, we can make $$A(t)=A_0\sin t$$

The generative process is such that

$$a_{lm}\sim\mathcal{N}(0,1)$$

for each $l$ and $m$. These $a_{lm}$ are the latent variables of the model.

### Election of content and style subspaces

Let's call $E_l$ the eigenspaces of the laplacian with eigenvalues $l(l+1)$ and let's make, arbitrarily

$$\mathcal{S}_c=E_1\oplus E_2$$
$$\mathcal{S}_s=E_3\oplus E_4$$

where $\oplus$ represents the direct sum of vector spaces.

_____________

## `DeformedSphere` class

This is a first attempt to create a class that models the deformation applied to a sphere, in the form of the spherical harmonics from above

In [ ]:
class DeformedSphere(trimesh.Trimesh):
    
    def __init__(self, reference_shape=vedo.Sphere().to_trimesh()):
        
        super().__init__(reference_shape.vertices, reference_shape.faces)        
        self.sphere_coords = self._appendSpherical_np(self.vertices)[:,3:]
                
    def deform(self, amplitude, l, m):
        
        '''
        This method returns a deformed sphere, using the spherical harmonic Ylm with an amplitude
        
        params:
            l, m: indices of the spherical harmonic (Ylm).
            amplitude: amplitude of the spherical harmonic.
        '''
        
        coefs = np.array([
            1 + amplitude * sph_harm(m, l, *self.sphere_coords[k, 1:3]).real
            for k in range(self.sphere_coords.shape[0])
        ])
        
        self.vertices = np.multiply(
            self.vertices, 
            np.kron(np.ones((3,1)), coefs).transpose()
        )
        
        return self
        
    # https://stackoverflow.com/questions/4116658/faster-numpy-cartesian-to-spherical-coordinate-conversion
    def _appendSpherical_np(self, xyz):
        
        '''
        params:
            xyz: NumPy array representing the (x,y,z) coordinates for a point cloud.
            
        return:
            A NumPy array with 6 columns containing the input (x,y,z) coordinates
            and, additionally, the (r, theta, phi) spherical coordinates        
        '''
        
        ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
        xy = xyz[:,0]**2 + xyz[:,1]**2        
        ptsnew[:,3] = np.sqrt(xy + xyz[:,2]**2)
        ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down
        #ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
        ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,0])
        return ptsnew
        

____________

In [11]:
N = 100
T = 50
sphere = vedo.Sphere(res=24).to_trimesh()
sphere_sph_coord = appendSpherical_np(sph.vertices)[:,3:]

In [12]:
mesh_list = []

for i in range(N):
    for t in range(T):
                                
        deformed_sphere = copy(sphere)
        
        amplitude_max = 2
        amplitude = amplitude_max * np.sin(2*np.pi*t/T)
            
        coefs = np.array([
            1 + amplitude * sph_harm(2, 2, *sphere_sph_coord[k,1:3]).real
            for k in range(sphere_sph_coord.shape[0])]
        )
            
        deformed_sphere.vertices = np.multiply(
            sph.vertices, 
            np.kron(np.ones((3,1)), coefs).transpose()
        )
        
        mesh_list.append(deformed_sphere)

NameError: name 'sphere_sph_coord' is not defined

In [ ]:
t = 15
mesh_list[t].show(faces=True)

In [ ]:
def f(l, m, amplitude):
    return DeformedSphere().deform(amplitude, l, m).show()

#### Saving the synthetic meshes as `obj` files

In [ ]:
odir = "/home/rodrigo/tmp/mallas_obj"
os.makedirs(odir, exist_ok=True)

N = 100

for i in range(N):
    for t, mesh in enumerate(mesh_list):
        
        # print(mesh.vertices)
        j = str(j)
        j = "0" * (2 - len(j)) + j # 5 --> 05, 15 --> 15
        
        with open(os.path.join(odir, "sphere_{}.obj".format(i)), "wt") as ff:        
            obj_content = trimesh.exchange.export.export_obj(
                mesh, 
                include_normals=False, 
                include_texture=False
            )
            ff.write(obj_content)
            
        # print(mesh.show())

__________________________

# Attempt to visualize meshes (to complete)

#### `itkwidgets`

In [ ]:
import itk
import itkwidgets
from itkwidgets import view

#### `Trimesh`

In [13]:
import trimesh

#### `pyvista`

In [3]:
import pyvista as pv

#### `polyscope`

In [2]:
import polyscope as ps
ps.init()
mesh = mesh_list[25]
ps.register_surface_mesh("sph", mesh.vertices, mesh.faces, smooth_shade=True)
ps.show()

NameError: name 'mesh_list' is not defined

________________________

________________________

In [ ]:
DeformedSphere().deform(0, 0, 0).vertices

In [ ]:
interact(
    f, 
    l=widgets.IntSlider(min=0,max=10), 
    m=widgets.IntSlider(min=-10,max=10), 
    amplitude=widgets.FloatSlider(min=0,max=2)
);

In [ ]:
[ sph_harm(2, 2, *sphere_sph_coord[k,1:3]) for k in range(sphere_sph_coord.shape[0]) ]

In [ ]:
interact(f, t=widgets.IntSlider(N_t-1, max=N_t-1));

In [ ]:
# np.polynomial.chebyshev.Chebyshev((1,2,3))()